In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
# os.chdir("../")


In [2]:
data=pd.read_csv("../merged/len3_prod.csv",encoding="shift-jis",index_col=0)
# whether count or not
count=0
# drop never treated group
data=data.dropna(subset=["release_year"])
data

,year,id,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,self_agg,...,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,date,release_year,成分,variable
3,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,NaN,NaN,NaN,1994.0,D00127,Piroxicam,1995/1/24,1995.0,ピロキシカム,id1
4,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,NaN,NaN,NaN,1994.0,D00132,Ketoprofen,1995/2/10,1995.0,ケトプロフェン,id1
5,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,NaN,NaN,NaN,2002.0,D01578,Pranoprofen,2004/2/16,2004.0,テルビナフィン塩酸塩,id1
6,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,NaN,NaN,NaN,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1
7,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,NaN,NaN,NaN,2010.0,D01709,Loxoprofen sodium hydrate,2011/1/21,2011.0,ロキソプロフェンナトリウム水和物,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,2021,625,抗ウイルス剤,259017253,130734846,22384111,335594,335594,4301,335594,...,143728443.0,260086.0,260086.0,2007.0,D00222,Aciclovir,2007/10/4,2007.0,アシクロビル,id2
3245,2021,625,抗ウイルス剤,259017253,130734846,22384111,335594,335594,4301,335594,...,143728443.0,260086.0,260086.0,2009.0,D06298,Vidarabine,2009/11/16,2009.0,ビダラビン,id1
3246,2021,629,その他の化学療法剤,50298237,35994690,20759707,0,0,0,0,...,18205617.0,0.0,0.0,2002.0,D02219,Terbinafine hydrochloride,2003/10/6,2003.0,プラノプロフェン,id2
3247,2021,629,その他の化学療法剤,50298237,35994690,20759707,0,0,0,0,...,18205617.0,0.0,0.0,2011.0,D00282,Clotrimazole,2011/5/10,2011.0,クロトリマゾール,id3


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elapsed"]=data["year"]-data["release_year"]
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
if count:
    aggregated_df = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()
else :
# dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first(), 
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,self_agg,...,elapsed_22,elapsed_23,elapsed_24,elapsed_25,elapsed_26,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31
0,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,24269268,19698580,19437160,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5239843,5044413,4568581,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008,131,眼科用剤,233484921,128902552,95963674,45459181,44792984,666197,0,...,0,0,0,0,0,0,0,0,0,0
4,2008,132,耳鼻科用剤,21365519,9339865,5619149,13521676,13521676,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2021,441,抗ヒスタミン剤,2170055,2170055,572374,1856437,1856437,312513,457791,...,0,0,0,0,0,1,0,0,0,0
360,2021,449,その他のアレルギー用薬,222931862,151727406,37317805,6767409,6626987,4370093,6661224,...,0,0,1,0,0,0,0,0,0,0
361,2021,625,抗ウイルス剤,259017253,130734846,22384111,335594,335594,4301,335594,...,0,0,0,0,0,0,0,0,0,0
362,2021,629,その他の化学療法剤,50298237,35994690,20759707,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
colnames

['elapsed_m14',
 'elapsed_m13',
 'elapsed_m12',
 'elapsed_m11',
 'elapsed_m10',
 'elapsed_m9',
 'elapsed_m8',
 'elapsed_m7',
 'elapsed_m6',
 'elapsed_m5',
 'elapsed_m4',
 'elapsed_m3',
 'elapsed_m2',
 'elapsed_m1',
 'elapsed_0',
 'elapsed_1',
 'elapsed_2',
 'elapsed_3',
 'elapsed_4',
 'elapsed_5',
 'elapsed_6',
 'elapsed_7',
 'elapsed_8',
 'elapsed_9',
 'elapsed_10',
 'elapsed_11',
 'elapsed_12',
 'elapsed_13',
 'elapsed_14',
 'elapsed_15',
 'elapsed_16',
 'elapsed_17',
 'elapsed_18',
 'elapsed_19',
 'elapsed_20',
 'elapsed_21',
 'elapsed_22',
 'elapsed_23',
 'elapsed_24',
 'elapsed_25',
 'elapsed_26',
 'elapsed_27',
 'elapsed_28',
 'elapsed_29',
 'elapsed_30',
 'elapsed_31']

connect generic usage

In [5]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric


,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.223357,0.058586
1,114.0,2009.0,0.223357,0.058586
2,114.0,2010.0,0.223357,0.058586
3,114.0,2011.0,0.223357,0.058586
4,114.0,2012.0,0.227087,0.058586
...,...,...,...,...
395,799.0,2019.0,0.006196,0.024777
396,799.0,2020.0,0.006056,0.022847
397,799.0,2021.0,0.007953,0.028184
398,799.0,2022.0,0.028110,0.043253


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,self_agg,...,elapsed_27,elapsed_28,elapsed_29,elapsed_30,elapsed_31,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,0,0,0,0,0,2008,35.35,114.0,0.223357,0.058586
1,2008,121,局所麻酔剤,24269268,19698580,19437160,0,0,0,0,...,0,0,0,0,0,2008,35.35,121.0,0.087367,0.047738
2,2008,123,自律神経剤,5239843,5044413,4568581,0,0,0,0,...,0,0,0,0,0,2008,35.35,123.0,0.188223,0.058512
3,2008,131,眼科用剤,233484921,128902552,95963674,45459181,44792984,666197,0,...,0,0,0,0,0,2008,35.35,131.0,0.163607,0.071454
4,2008,132,耳鼻科用剤,21365519,9339865,5619149,13521676,13521676,0,0,...,0,0,0,0,0,2008,35.35,132.0,0.045752,0.059251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2170055,2170055,572374,1856437,1856437,312513,457791,...,1,0,0,0,0,2021,79.00,441.0,0.342768,0.263456
346,2021,449,その他のアレルギー用薬,222931862,151727406,37317805,6767409,6626987,4370093,6661224,...,0,0,0,0,0,2021,79.00,449.0,0.608051,0.363450
347,2021,625,抗ウイルス剤,259017253,130734846,22384111,335594,335594,4301,335594,...,0,0,0,0,0,2021,79.00,625.0,0.502607,0.052158
348,2021,629,その他の化学療法剤,50298237,35994690,20759707,0,0,0,0,...,0,0,0,0,0,2021,79.00,629.0,0.284349,0.073842


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,self_agg,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2008,114,解熱鎮痛消炎剤,125297602,123316655,94834673,32453698,32431251,22447,0,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,24269268,19698580,19437160,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5239843,5044413,4568581,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008,131,眼科用剤,233484921,128902552,95963674,45459181,44792984,666197,0,...,0,0,0,0,0,0,0,0,0,0
4,2008,132,耳鼻科用剤,21365519,9339865,5619149,13521676,13521676,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2170055,2170055,572374,1856437,1856437,312513,457791,...,0,0,0,0,0,1,0,0,0,0
346,2021,449,その他のアレルギー用薬,222931862,151727406,37317805,6767409,6626987,4370093,6661224,...,0,0,0,0,0,0,1,0,0,0
347,2021,625,抗ウイルス剤,259017253,130734846,22384111,335594,335594,4301,335594,...,0,0,0,0,0,0,0,1,0,0
348,2021,629,その他の化学療法剤,50298237,35994690,20759707,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
# dummy ver
if count==0:
    df_with_dummies.to_csv("../merged/len3_prod_DN.csv",encoding="shift-jis")
else:
# count ver
    df_with_dummies.to_csv("../merged/len3_prod_sum_DN.csv",encoding="shift-jis")